In [1]:
#importing packages
import json
import re
import pandas as pd

In [ ]:
def extract_json_from_html(html_path, save_output=False):
    html_file = open(html_path, 'r', encoding = "utf8")
    html = html_file.read()
    html_file.close()
    regex_pattern = r'(?<=require\.config\.params\["args"\].=.)[\s\S]*?;'
    data_txt = re.findall(regex_pattern, html)[0]

    # add quotations for json parser
    data_txt = data_txt.replace('matchId', '"matchId"')
    data_txt = data_txt.replace('matchCentreData', '"matchCentreData"')
    data_txt = data_txt.replace('matchCentreEventTypeJson', '"matchCentreEventTypeJson"')
    data_txt = data_txt.replace('formationIdNameMappings', '"formationIdNameMappings"')
    data_txt = data_txt.replace('};', '}')

    if save_output:
        # save json data to txt
        output_file = open(f"{html_path}.txt", "wt")
        n = output_file.write(data_txt)
        output_file.close()

    return data_txt

In [ ]:
def extract_data_from_dict(data):
    # load data from json
    event_types_json = data["matchCentreEventTypeJson"]
    formation_mappings = data["formationIdNameMappings"]
    events_dict = data["matchCentreData"]["events"]
    teams_dict = {data["matchCentreData"]['home']['teamId']: data["matchCentreData"]['home']['name'],
                  data["matchCentreData"]['away']['teamId']: data["matchCentreData"]['away']['name']}
    players_dict = data["matchCentreData"]["playerIdNameDictionary"]
    # create players dataframe
    players_home_df = pd.DataFrame(data["matchCentreData"]['home']['players'])
    players_home_df["teamId"] = data["matchCentreData"]['home']['teamId']
    players_away_df = pd.DataFrame(data["matchCentreData"]['away']['players'])
    players_away_df["teamId"] = data["matchCentreData"]['away']['teamId']
    players_df = pd.concat([players_home_df, players_away_df])
    players_ids = data["matchCentreData"]["playerIdNameDictionary"]
    return events_dict, players_df, teams_dict

In [ ]:
#Getting data for Nantes vs PSG from WhoScored
match_html_path = "Nantes 0-2 Paris Saint-Germain - Ligue 1 2023_2024 Live.htm"
json_data_txt = extract_json_from_html(match_html_path)
data = json.loads(json_data_txt)
events_dict, players_df, teams_dict = extract_data_from_dict(data)

In [ ]:
def get_events_df(events_dict, players_df):
    events = pd.DataFrame(events_dict)
    events['eventType'] = events.apply(lambda row: row['type']['displayName'], axis=1)
    events['outcomeType'] = events.apply(lambda row: row['outcomeType']['displayName'], axis=1)
    events['period'] = events.apply(lambda row: row['period']['displayName'], axis = 1)
    events['type'] = events.apply(lambda row: row['type']['displayName'], axis=1)
    
    events = events.merge(players_df[['playerId', 'name', 'shirtNo']], on='playerId', how='left')
    
    return events

In [ ]:
events_df = get_events_df(events_dict, players_df)

In [ ]:
#exporting the dataframe
events_df.to_csv('NANPSG.csv')